# UBS_lauzhack - Entity resolution model

## 1. Pre-processing

Observed the given dataset and apply the proper transformations to the data in order to better anlyse it and create the model.

In [9]:
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
import warnings


InteractiveShell.ast_node_interactivity = "all"
#pd.set_option('precision', 3)
display.precision = 3
sns.set()
warnings.filterwarnings('ignore')

# extra imports
from pandas import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
from statsmodels.genmod.generalized_linear_model import GLM
from pandas.plotting import scatter_matrix
from scipy.stats import boxcox



In [19]:
# Read the CSV files from the 'data' subfolder
booking_data = pd.read_csv('data/account_booking_train.csv')
parties_data = pd.read_csv('data/external_parties_train.csv')

In [20]:
# Combine the tables with merge using 
combined_data = pd.merge(booking_data, parties_data, on='transaction_reference_id', how='inner')

In [21]:
combined_data.isnull().sum()

transaction_reference_id            0
debit_credit_indicator              0
account_id                          0
transaction_amount                  0
transaction_currency                0
transaction_date                    0
party_role                          0
party_info_unstructured             0
parsed_name                         0
parsed_address_street_name        739
parsed_address_street_number     3030
parsed_address_unit             11064
parsed_address_postal_code       3480
parsed_address_city              1168
parsed_address_state             9107
parsed_address_country           6490
party_iban                       3399
party_phone                      5000
external_id                         0
dtype: int64